# Main

In [1]:
import numpy as np
from functions import Affine, ReLU, get_epsilon, get_stop, create_constraints_dictionaries, update_layer_0, check_activation_functions, create_extra_layer, calculate_previous_layer_number, get_correctly_classified_pictures, update_upper_lower, propagate, sum_arrays, check_specification

### Create dictionaries for weights, biases and activation functions.

In [2]:
from json import loads

# File containing the network.
net_file = open("custom_relu.tf", mode='r')
net_file_list = list(net_file)

# Empty lists are created for the weights, biases and activation functions.
weights_list = []
biases_list = []
activations_list = []

# weights_list, activations_list and biases_list are updated dinamically by reading the lines of the network file.
for i in range(0, (len(net_file_list) - 2), 3):
    activations_list.append(net_file_list[i][:-1])
    weights_list.append(net_file_list[i+1][:-1])
    biases_list.append(net_file_list[i+2][:-1])

# Check that all activation functions are either ReLU or Affine.
check_activation_functions(activations_list)
    
# The lists become dictionaries, where layer_i is the key for the array of weights/biases/activations for that layer.
weights = {}
biases = {}
activations = {}
# Iterate over all layers.
for i in range(0, len(weights_list)):
    weights["layer%s" %(i + 1)] = np.array(loads(weights_list[i])) # json.loads turns the string into a (nested) list.
    biases["layer%s" %(i + 1)] = np.array(loads(biases_list[i]))
    activations["layer%s" %(i + 1)] = activations_list[i]

### Create data structures for the input and output .

In [3]:
from pandas import *

dataset = "example.csv"

# The picture, stored in the csv file, is read and saved in pic. It is of type _io.TextIOWrapper.
pic = open(dataset, mode='r')

# input_values is a list of strings, containing the first line of the csv file.
picture1 = (pic.readline()).rstrip().split(",")

# Create a list with the first column of the csv file.
data = read_csv(dataset)
output_column = data[picture1[0]].tolist()
# Prepend the first element to the list, since it was lost. Transform output_column in an array.
output_column.insert(0, int(picture1[0]))
output_column = np.array(output_column)

# Remove the first element from input_values, since it is the output label from the first line.
picture1.pop(0)

# Create the input dictionary

input_dic = {}
counter = 1
input_dic["picture1"] = np.array(list(map(int, picture1)))/255 # / 255 to normalize.

# All lines (minus their first element, which is the output) are read as strings.
for line in pic.readlines():
    
    counter = counter + 1
    # Lines become arrays of integers and are put in the dictionary as values of the respective picture (which is the key).
    input_dic["picture%s" %(counter)] = np.array(list(map(int, (line.rstrip().split(",")[1:]))))/255 # /255 to normalize.

### Calculate the output of the network for each picture and the percentage of pictures it classifies correctly.

In [4]:
# Create an array of flags. 1 (0) will mean that the picture corresponding to that index was (not) classified correctly.
correctly_classified = np.zeros(len(output_column), dtype = int)

# Iterate over all pictures.
for picture in input_dic:
    
    outputarray = input_dic[picture]
    
    # Calculate the output of the neural network for the current picture.
    for layer in weights:
        if (activations[layer] == "Affine"):
            outputarray = Affine(outputarray, weights[layer], biases[layer])
        else:
            outputarray = ReLU(outputarray, weights[layer], biases[layer])
            
    # If the current picture was classified correctly, update the array "correctly_classified".
    # To do this, check whether the greatest element of "outputarray" is at the index equal to the label of the respective picture.
    if (np.amax(outputarray) == outputarray[output_column[int([(s) for s in picture.split("e") if s.isdigit()][0]) - 1]]):
        correctly_classified[int([(s) for s in picture.split("e") if s.isdigit()][0]) - 1] = 1

# Calculate the percentage of correctly classified pictures.
correctness_basic = ((np.sum(correctly_classified, dtype = int)) / correctly_classified.size) * 100

### Get inputs from the user and do some preparations.

In [5]:
# Have the user provide an ε value, determining the length of the noise interval centred on each pixel.
epsilon = get_epsilon()

# Have the user provide the number of the layer at which they want to use the concrete bounds when calculating
# new upper and lower bounds.
stop = get_stop()

# Work out the dictionary of correctly classified pictures with the correct output neuron.
correctly_classified_pictures = get_correctly_classified_pictures(correctly_classified, output_column)

correct_array = np.zeros(len(correctly_classified_pictures), dtype = int)

The length of the noise interval centred on each pixel will be 2ε. Please, write a value for ε: 1
Please, write the number of the layer you want to stop and take the concrete bounds at when calculating the new upper and lower bounds. If you want maximum precision and thus to select the concrete bounds at the input layer, you can write 0. If an invalid number is provided, it will be truncated to make it integer.0


### Perform abstract analysis.

In [6]:
counter = -1
from copy import deepcopy

# Loop over all correctly classified pictures.
for picture in correctly_classified_pictures:
    
    # Create dictionaries for polyhedral constraints and upper and lower bounds.
    constraints = create_constraints_dictionaries(weights, activations)
    upper_lower_bounds = constraints[0]
    poly_constraints = constraints[1]
    # Given a picture, fill in the values for the input layer in the dictionary of upper and lower bounds.
    update_layer_0(picture, input_dic, upper_lower_bounds, epsilon)
    
    # Calculate upper and lower bounds and polyhedral constraints for all neurons in all layers.
    for layer in weights:       
        propagate(layer, weights, poly_constraints, biases, upper_lower_bounds, activations, stop)
    
    counter = counter + 1
    
    # Check the specification.
    check_specification(upper_lower_bounds, correctly_classified_pictures, picture, counter, correct_array, poly_constraints, stop)        

### Print the results.

In [8]:
print(f"Number of pictures classified correctly before the analysis: {np.sum(correctly_classified)}")
print("Number of pictures classified correctly after the analysis:", np.sum(correct_array, dtype = int))
print("Percentage of pictures classified correctly after the analysis, over the pictures that " +\
      f"were classified correcly before it: {((np.sum(correct_array, dtype = int)) / correct_array.size) * 100}%")

Number of pictures classified correctly before the analysis: 1
Number of pictures classified correctly after the analysis: 1
Percentage of pictures classified correctly after the analysis, over the pictures that were classified correcly before it: 100.0%
